<a href="https://colab.research.google.com/github/dtabuena/EphysLib/blob/main/analyze_rheobase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
''' Requires: 
    Simple_ABF_tools.ipynb
    Basic_Ephys.ipynb
'''

def rheobase_analyzer(file_name,
                        spike_args =  {'spike_thresh':20, 'high_dv_thresh': 25,'low_dv_thresh': -5,'window_ms': 2},
                        to_plot=False,
                        verbose=False,
                        single_spike=True):
    
    ''' File Analyzer for Rheobase etc  '''

    results = {} # default return

    abf = abf_or_name(file_name)
    if to_plot: plot_sweeps_and_command(abf)

    # Rheobase Measure:
    if len(abf.sweepList)<2:
        return results
    else:
        is_base, is_stim = protocol_baseline_and_stim(abf)
        stim_currents, spike_counts, spike_rates, V_before_AP,V_before_stim,_ = spikes_per_stim(abf, spike_args,thresh=20,to_plot=to_plot)
        single_spikes = spike_counts==1
        zero_spikes = spike_counts==0
        if single_spike:
            none_to_one = np.full(single_spikes.shape, False)
            none_to_one[1:] = np.logical_and(single_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_one)[0]
        else:
            some_spikes = spike_counts>0
            none_to_some = np.full(single_spikes.shape, False)
            none_to_some[1:] = np.logical_and(some_spikes[1:], zero_spikes[:-1])
            first_spike_stim = np.where(none_to_some)[0]


    if first_spike_stim.size == 0:
        return results
    else:
        if first_spike_stim.size >1:
            first_spike_stim = np.min(first_spike_stim)
        results['Rheobase'] = stim_currents[first_spike_stim]
        results['Vhold_spike'] = V_before_stim[first_spike_stim]
        results['AP_thresh'] = V_before_AP[first_spike_stim]

    if first_spike_stim.size>0:
        try:
            abf = abf_or_name(file_name)
            abf.setSweep(first_spike_stim[0])
            ap_params = single_ap_stats(abf.sweepX,abf.sweepY,abf.sampleRate,to_plot=True,verbose=True)
            results.update(ap_params)
        except:
            print('ap_stats_failed: ', file_name)


    return results




In [ ]:
abf